No exercicio 2 é nos pedido para implementar o NTT-CRT tal como é descrito nos apontamentos fornecidos pelo docente.

O 1º passo para isto é descobrir um N ,que seja potencia de 2, e um primo que  verifique $$\,q \equiv 1 \bmod 2N\,$$ 
É de importante relembrar que o N escolhido tem de ser sufientemente grande para que $$\,\mathcal{R}_{q,N}\,$$ contenha todos os polinómios que, previsivelmente, são relevantes à aplicação desta tecnica

In [1]:
n = 2048 #2^32
q = next_prime(2 * n)
i = 1
while q % (2*n) != 1:
    q = next_prime(n + i) 
    i += 1


Após descobertos podemos então gerar as variaveis necessárias para a execução deste algoritmo.
Iremos então criar um corpo baseado no primo calculado préviamente, $\,\mathbb{F}_q\,$. A partir deste corpo somos capazes de gerar o polinómio do tipo $\,^N + 1$. Com isto podemos utilzar uma das suas raizes, $\xi$ ,  para calcular $\,\omega,$, uma vez que $\,\omega \equiv  \xi^2\,$

In [2]:
F = GF(q)
R = PolynomialRing(F, name="w")
w = R.gen()
g = (w^n + 1)
xi = g.roots(multiplicities=False)[-1]
rs = [xi^(2*i+1)  for i in range(n)] 
base = crt_basis([(w - r) for r in rs])  

f = R.random_element(1023)

u = f.list()
preenchido = u + [0]*(n-len(u)) 

def ntt(xi,N,f):
    if N==1:
        return f
    N_ = N/2 ; xi2 =  xi^2  
    f_plus = [f[2*i]   for i in range(N_)] ; f_minus = [f[2*i+1] for i in range(N_)] 
    ff_plus = ntt(xi2,N_,f_plus) ; ff_minus = ntt(xi2,N_,f_minus)  
    s  = xi ; ff = [F for i in range(N)] 
    for i in range(N_):
        a = ff_plus[i] ; b = s*ff_minus[i]  
        ff[i] = a + b ; ff[i + N_] = a - b 
        s = s * xi2                     
    return ff

ff = ntt(xi,n,preenchido)
fff = sum([ff[i]*base[i] for i in range(n)])
print(f == fff)

True
